# Classification of Unstructured Documents 
## *Transfer Learning with BERT*
### GRAD-E1394 Deep Learning

---

Authors:
*   Ma. Adelle Gia Arbo, m.arbo@students.hertie-school.org
*   Janine De Vera, j.devera@students.hertie-school.org
*   Lorenzo Gini, l.gini@students.hertie-school.org
*   Lukas Warode, l.warode@students.hertie-school.org | lukas.warode@gmx.de

---

This tutorial demonstrates the pipeline for classifying unstructured documents, particularly those in PDF format. 


# Table of Contents


*   [Memo](#memo)
*   [Overview](#overview)
*   [Background & Prerequisites](#background-and-prereqs)
*   [Software Requirements](#software-requirements)
*   [Data Description](#data-description)
*   [Methodology](#methodology)
*   [Results & Discussion](#results-and-discussion)
*   [References](#references)


<a name="memo"></a>
# Memo

Write a memo for the leadership explaining in layman's terms why this topic is relevant for public policy. Discuss relevant research works, real-world examples of successful applications, and organizations and governments that apply such approaches for policy making. 


### *Sketch Lukas (not done)*

#### Main Point / Take Away (Executive Summary???)

#### Background 

Companies as well as public and political institutions are able and ofent required to access loads of information nowadays. In doing that, they often face the problem of unstructured information, especially when dealing with text data. Data in textual form is the most common type of unstructured, unfortunately, it is also representing the most fundamental type of documents with respect to policymakers and public institutions: Legal documents, bills and policy papers are just some examples of common text sources, which are part of the daily business in the political world. Unstructured text data entails a variety of different problems when it comes actually gaining quantitative analytical insights from them. Computers commonly have difficulties understanding textual data. Analytical and technical competences are also scarce: Only 18% of companies are able to use unstructured data, while most organizations are make their (data-driven) decisions on the basis of only 10 to 20% of their available data source. The situation for public institutions is worsend by the fact that most of the modern text data analysis frameworks and models are dominated by different industry players, while being designed and trained according to the industry-specific needs, which do not transfer to the needs of the mentioned political text sources. The EU Commission (or directly mention DMA?) …

This motivates the necessity of implementing …

#### Evidence (Sktech our technical solution?)
- Text Classification Pipepline + BERT Model(ling)

- Results??


#### Conclusion and Implementation

- Results??

- Being specific about implementation and actual usage (maybe even specify DMA context a bit more and say who and what stakeholders/institutions are taking what position in context)

*Open points*

- Include practicality and gained efficiency
- Normative perspective: "Technical" objectivity (but one needs to be careful for potential biases)

\
- Directly mention DMA at beginning?
- How structured should the memo be?

\
- Sources need to be added for numbers of slides (e.g. 18% of companies blabla)


*Structure*
![](https://mitcommlab.mit.edu/broad/wp-content/uploads/sites/5/2017/12/policy-memo-struct-700x843.png)

![](https://www.bu.edu/sph/files/2014/06/Policy-Memo-Organization.jpg)

<a name="overview"></a>
# Overview

Over 80% of all data is *unstructured*. Most of the information we consume comes in a format that is not organized in a pre-defined manner (e.g. tables) or with a specific data model in mind (e.g. matrices). 

<u>Text</u> is the most common type of unstructured data and it comes in a variety of forms, like blogs, news articles, social media content, as well as official documents. This lack of structure that can be readily understood by machines makes it difficult to maximize text as a data source. Algorithms that efficiently and accurately process text would have a variety of applications in organisations, especially in public institutions. 

In this tutorial we demonstrate one such application in the context of the European Commission (EC). Whenever new legislation is proposed, the EC opens **public consultations** where various stakeholders (e.g. businesses, academia, law firms, associations, private individuals) submit documents that detail their views on the proposal. The EC receives anywhere between 10,000 to 4 million of these public consultation documents annually. Using machine learning and deep learning methods to process these documents would streamline the Commission's review of stakeholder comments, which consequently allows them to integrate more information into their policymaking process. 

The main goal of this tutorial is to walk you through the steps of building a <u>document classifier</u> using a deep learning model called **Bidirectional Encoder Representations from Transformers (BERT)**. By the end of this tutorial you will understand how to:  

> 1. Extract, clean, and pre-process information from PDF documents
> 2. Use the pre-processed text as input to machine learning/deep learning models
> 3. Build a text/document classifier with BERT 
> 4. Compare BERT with text classifiers built using other models

We will then apply these learnings to accomplish a research objective: 

 > Classify public consultation documents of the recently enacted **Digital Markets Act** according to whether a stakeholder **agrees or disagrees** with the DMA proposal. 

The Digital Markets Act is a regulation in the European Union which came into force this 2022. It aims to promote fair competition within the digital market by defining rules for “gatekeepers” or large online platforms. Majority of the public consultation documents submitted for the DMA are from companies and business associations who will likely be affected by the law.

<a name="background-and-prereqs"></a>
# Background & Prerequisites

For this tutorial, you would need to be familiar with object oriented programming, common python libraries such as *numpy* and *pandas*, and libraries used for model building, such as *scikit-learn* and *pytorch*. Working knowledge of common language processing concepts (e.g. stemming, lemmatization, TF-IDF) would also be useful. 

Important concepts that need to be explained briefly:  

## Reading materials

For detailed explanations of the topics covered in this tutorial, below is a list helpful references.

**Basics of natural language processing models:**
* Vajjala, S., Majumder, B., Gupta, A., & Surana, H. (2020). Practical natural language processing: a comprehensive guide to building real-world NLP systems. O'Reilly Media.


**BERT:**
* Horev, R. (2018). BERT Explained: State of the art language model for NLP. Towards Data Science, 10.


<a name="software-requirements"></a>
# Software Requirements
To install software requirments and dependencies, please create a new environment using the *environment.yml* file which accompanies this notebook. 

In [ ]:
!conda env create -f environment.yml

In [ ]:
# Data visualization
import matplotlib.pyplot as plt 

# Data manipulation
import pandas as pd
import numpy as np

<a name="data-description"></a>
# Data Description

As mentioned in the [Overview](#overview), the methods discussed in this tutorial will be applied to public consultation documents of the **Digital Markets Act (DMA)**. The open public consultation for DMA received **188 responses**. In addition to document submissions, each respondent answered an accompanying survey. They were asked a series of questions about what they think about the law. 

To build our document classifier we need the raw **text** from the public consultation submissions, and a corresponding **label** for each document which indicates whether or not the author/s agree or disagree with the DMA proposal. 

**Text:**
The submissions come in the form of **pdf files**. These documents need to be parsed in order to extract raw text. 

**Labels:**
The labels are extracted from the **survey**, specifically from the question: 

*"Do you consider that there is a need for the Commission to be able to intervene in gatekeeper scenarios to prevent/address structural competition problems?*
 

## Data Download

The pdf documents and the survey used to generate labels can be downloaded from the <a href="https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12416-New-competition-tool/public-consultation_en">DMA consultation page</a>. 

## Data Preprocessing
Additionally, you can include any data preprocessing steps and exploratory data analyses (e.g. visualize data distributions, impute missing values, etc.) in this section to allow the users to better understand the dataset. 

In this section, you might also want to describe the different input and output variables, the train/val/test splits, and any data transformations.

(Describe processing of labels here)

In [ ]:
# Insert data pre-processing and exploratory data analysis
# code here. Feel free to break this up into several code
# cells, interleaved with explanatory text. 

<a name="methodology"></a>
# Methodology

This section of the tutorial is a step-by-step walkthrough of text classification pipeline, using DMA public consultation documents as described above. Below is an outline of this section: 

<ol type="A">
  <li>Data Preparation </li>
  <li>Text Representation</li>
  <li>Model Training (with hyperparameter tuning)</li>
  <ol>
    <li> Training baseline models (logistic, XGBoost)
    <li> Training a DL classifiers (LSTM)
    <li> Training a base BERT model
    <li> Training BERT variants
  </ol>
  <li> Model Evaluation
</ol>

## A. Data Preparation 
Parsing PDF files, cleaning text, pre-processing 

## B. Text Representation
Since this step is built inside BERT and other DL models, we can use what Lorenzo has done for the baseline models (TF-IDF & embeddings) to illustrate how text representation works. 

In the BERT section explain how the architecture processes text to create embeddings. 

## C. Model Training (with hyperparameter tuning)

### C.1 Baseline Models

##### Logistic Regression

##### Gradient Boosting

### C.2 Long Short-Term Memory Network

### C.3 Base BERT Model

### C.4 Other BERT Variants

## D. Model Evaluation
Including comparison of models

<a name="results-and-discussion"></a>
# Results & Discussion

In this section, describe and contextualize the results shown in the tutorial. Briefly describe the performance metrics and cross validation techniques used. 

Finally, include a discussion on the limitations and important takeaways from the exercise.

## Limitations
*   The tutorial is focused on education and learning. Explain all the simplifications you have made compared to applying a similar approach in the real world (for instance, if you have reduced your training data and performance).
*   ML algorithms and datasets can reinforce or reflect unfair biases. Reflect on the potential biases in the dataset and/or analysis presented in your tutorial, including its potential societal impact, and discuss how readers might go about addressing this challenge. 

## Next Steps
*   What do you recommend would be the next steps for your readers after finishing your tutorial?
*   Discuss other potential policy- and government-related applications for the method or tool discussed in the tutorial.
*   List anything else that you would want the reader to take away as they move on from the tutorial.

<a name="references"></a>
# References

Include all references used. 

For example, in this template:

*   EarthCube Notebook Template: https://github.com/earthcube/NotebookTemplates
*   Earth Engine Community Tutorials Style Guide: https://developers.google.com/earth-engine/tutorials/community/styleguide#colab
*   Google Cloud Community Tutorial Style Guide: https://cloud.google.com/community/tutorials/styleguide
*   Rule A, Birmingham A, Zuniga C, Altintas I, Huang S-C, Knight R, et al. (2019) Ten simple rules for writing and sharing computational analyses in Jupyter Notebooks. PLoS Comput Biol 15(7): e1007007. https://doi.org/10.1371/journal.pcbi.1007007




## Acknowledgement

These guidelines are heavily based on the Climate Change AI template for the for the tutorials track at the [NeurIPS 2021 Workshop on Tackling Climate Change with Machine Learning](https://www.climatechange.ai/events/neurips2021). 